In [1]:
# CS524 NLP - Project 2: 
# Binary Authorship Attribution G.K. Chesterton using BERT
# Team 7: Zack Malkmus, Tyler Nitzsche, Andrew Meuller, Gabriel Laboy
#
# TO RUN:
#   1. Install jupyter notebooks
#   2. 'pip install -r requirements.txt'
#   3. Run the code in the jupyter notebook

In [2]:
# --------------------------------------------
# Import Libraries
# --------------------------------------------

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch.nn as nn
import torch.optim as optim

import time
import datetime


/home/zmalk/conda/envs/nlp/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# --------------------------------------------
# Reading Dataset
# --------------------------------------------

df = pd.read_csv('text_to_authorship.csv')

print("First 5 entries:")
print(df.head())

print("\nNull values in each column:")
print(df.isnull().sum())


First 5 entries:
                                                text  label
0  \n“But why Turkish?” asked Mr. Sherlock Holmes...      0
1  \nOf all the problems which have been submitte...      0
2  Valentin, Chief of the Paris Police, was late ...      1
3  \nOn glancing over my notes of the seventy odd...      0
4  \n    "Monsieur Arsène Lupin has the honour to...      0

Null values in each column:
text     0
label    0
dtype: int64


In [4]:
# --------------------------------------------
# Data Preprocessing
# --------------------------------------------

df = df.dropna()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'],
    df['label'],
    test_size=0.2,
    random_state=42
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [5]:
# --------------------------------------------
# Dataset and DataLoaders
# --------------------------------------------

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
            
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=False,  # We don't need attention masks
            return_tensors='pt',
        )
            
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [6]:
# --------------------------------------------
# Model Setup
# --------------------------------------------

class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes, kernel_sizes, num_filters, dropout=0.5):
        super(CNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (K, embed_size)) for K in kernel_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)
        
    def forward(self, x):
        x = self.embedding(x)  # (batch_size, seq_len, embed_size)
        x = x.unsqueeze(1)     # (batch_size, 1, seq_len, embed_size)
        x = [nn.functional.relu(conv(x)).squeeze(3) for conv in self.convs]  # [(batch_size, num_filters, seq_len - K + 1)]
        x = [nn.functional.max_pool1d(i, i.size(2)).squeeze(2) for i in x]   # [(batch_size, num_filters)]
        x = torch.cat(x, 1)    # (batch_size, num_filters * len(kernel_sizes))
        x = self.dropout(x)
        logits = self.fc(x)
        return logits

# Parameters
vocab_size = tokenizer.vocab_size
embed_size = 128
num_classes = 2
kernel_sizes = [3, 4, 5]
num_filters = 100
dropout = 0.5

# Initialize model
model = CNNTextClassifier(vocab_size, embed_size, num_classes, kernel_sizes, num_filters, dropout)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


CNNTextClassifier(
  (embedding): Embedding(30522, 128)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 128), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 128), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 128), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=300, out_features=2, bias=True)
)

In [7]:
# --------------------------------------------
# Training
# --------------------------------------------

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in data_loader:
        optimizer.zero_grad()
            
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
            
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
            
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(data_loader)
    return avg_loss

epochs = 3

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    print(f'Training loss: {train_loss:.4f}')


Epoch 1/3
Training loss: 0.7545
Epoch 2/3
Training loss: 0.2893
Epoch 3/3
Training loss: 0.1764


In [8]:
# --------------------------------------------
# Evaluation
# --------------------------------------------

def eval_model(model, data_loader, device):
    model.eval()
    predictions, true_labels = [], []
        
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
                
            outputs = model(input_ids)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
        
    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, target_names=['Other Authors', 'G.K. Chesterton'])
    cm = confusion_matrix(true_labels, predictions)
    return accuracy, report, cm

accuracy, report, cm = eval_model(model, val_loader, device)
print(f'Validation Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)
print('Confusion Matrix:')
print(cm)


Validation Accuracy: 0.9756
Classification Report:
                 precision    recall  f1-score   support

  Other Authors       0.96      1.00      0.98        24
G.K. Chesterton       1.00      0.94      0.97        17

       accuracy                           0.98        41
      macro avg       0.98      0.97      0.97        41
   weighted avg       0.98      0.98      0.98        41

Confusion Matrix:
[[24  0]
 [ 1 16]]
